In [2]:
import cv2
import pandas as pd
import numpy as np
import os

from skimage.feature import greycomatrix, greycoprops

import warnings

warnings.filterwarnings('ignore')

In [3]:
# Creating array

images = []
labels = []

#Define dataset
Data_path = "./Dataset/Resized/"

#Define Class labels and corresponding Folder names

class_labels = ['Bacterial', 'Fungal', 'Healthy', 'Hypersensitive']
class_folders = ['Bacterial', 'Fungal', 'Healthy', 'Hypersensitive']

#Appending all images


for i in class_folders:
    count = 0
    for filename in os.listdir(Data_path + i):
        # Read the image file directly in grayscale mode
        img = cv2.imread(Data_path + i + '/' + filename, cv2.IMREAD_GRAYSCALE)

        # Check if the image is valid
        if img is not None:
            images.append(img)
            labels.append(class_folders.index(i))
            count += 1
            if count == 500:
                break
            
    print(i + ' : ' + str(count))



#Importing images in numpy arrey

images = np.array(images)
lables = np.array(labels)

Bacterial : 241
Fungal : 210
Healthy : 280
Hypersensitive : 210


In [4]:
images.shape

(941, 1080, 1920)

In [5]:
lables.shape

(941,)

In [6]:

# ----------------- calculate greycomatrix() & greycoprops() for angle 0, 45, 90, 135 ----------------------------------
def calc_glcm_all_agls(img, label, props, dists=[5], agls=[0, np.pi/4, np.pi/2, 3*np.pi/4], lvl=256, sym=True, norm=True):
    
    glcm = greycomatrix(img, distances=dists, angles=agls, levels=lvl, symmetric=sym, normed=norm)
    
    feature = []

    glcm_props = [propery for name in props for propery in greycoprops(glcm, name)[0]]
    for item in glcm_props:
            feature.append(item)
    feature.append(label) 
    
    return feature

In [7]:
# ----------------- call calc_glcm_all_agls() for all properties ----------------------------------
properties = ['dissimilarity', 'correlation', 'homogeneity', 'contrast', 'ASM', 'energy']

glcm_all_agls = []

for img, label in zip(images, labels): 

    glcm_all_agls.append(
         
            calc_glcm_all_agls(img, label, props=properties)

                        )
 

In [8]:
features = np.array(glcm_all_agls)

In [9]:
features.shape

(941, 25)

In [10]:
columns = []

angles = ['0', '45', '90','135']

for name in properties :

    for ang in angles:

        columns.append(name + "_" + ang)
        
columns.append("label")

In [11]:
columns

['dissimilarity_0',
 'dissimilarity_45',
 'dissimilarity_90',
 'dissimilarity_135',
 'correlation_0',
 'correlation_45',
 'correlation_90',
 'correlation_135',
 'homogeneity_0',
 'homogeneity_45',
 'homogeneity_90',
 'homogeneity_135',
 'contrast_0',
 'contrast_45',
 'contrast_90',
 'contrast_135',
 'ASM_0',
 'ASM_45',
 'ASM_90',
 'ASM_135',
 'energy_0',
 'energy_45',
 'energy_90',
 'energy_135',
 'label']

In [12]:
df = pd.DataFrame(features)

In [13]:
df[:0]

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24


In [14]:
df.to_csv('test.csv', header=None, index=None)

In [15]:
df = pd.read_csv('test.csv')

In [16]:
features = np.array(df)

In [17]:
np.save("GLCM_features.npy", features)

In [18]:
np.save("Lables.npy", lables)

In [19]:
features.shape

(940, 25)

In [20]:
df.shape

(940, 25)